# **Project Name**    - Integrating MLOps for Predictive and Recommender Systems in Travel/ Hotel_Recommender_System/ streamlit_app

##### **Project Type**    - Recommender System
##### **Contribution**    - Individual

# **Problem Statement**


Build a recommendation model to provide hotel suggestions based on user preferences and historical data. Develop a Streamlit web application to display insights and visualizations derived from the deployed travel recommendation model, offering an interactive and user-friendly interface for data exploration.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Import Libraries
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import streamlit as st
import random

import warnings
warnings.filterwarnings("ignore")

### Dataset Loading

In [ ]:
# Load Dataset
url1 = 'https://raw.githubusercontent.com/satyamkulkarni122/Integrating-MLOps-for-Predictive-and-Recommender-Systems-in-Travel/main/flights.csv'
url2 = 'https://raw.githubusercontent.com/satyamkulkarni122/Integrating-MLOps-for-Predictive-and-Recommender-Systems-in-Travel/main/hotels.csv'
url3 = 'https://raw.githubusercontent.com/satyamkulkarni122/Integrating-MLOps-for-Predictive-and-Recommender-Systems-in-Travel/main/users.csv'
flights_df = pd.read_csv(url1)
hotel_df = pd.read_csv(url2)
users_df = pd.read_csv(url3)
sample_size = 5000

In [ ]:
# Set a random seed for reproducibility
random.seed(42)

In [ ]:
users_with_enough_interactions_df = hotel_df.groupby(['userCode']).size().groupby('userCode').size()

In [ ]:
users_interactions_count_df = hotel_df.groupby(['userCode','name']).size().groupby('userCode').size()
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 2].reset_index()[['userCode']]

In [ ]:
interactions_from_selected_users_df = hotel_df.merge(users_with_enough_interactions_df,
               how = 'right',
               left_on = 'userCode',
               right_on = 'userCode')

In [ ]:
# Encode userCode and hotel name to numeric values
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#df_hotel['userCode'] = label_encoder.fit_transform(df_hotel['userCode'])
interactions_from_selected_users_df['name_encoded'] = label_encoder.fit_transform(interactions_from_selected_users_df['name'])

In [ ]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [ ]:
interactions_full_df = interactions_from_selected_users_df.groupby(['name_encoded','userCode'])['price'].sum().reset_index()

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                                               stratify=interactions_full_df['userCode'],
                                   test_size=0.25,
                                   random_state=42)

x_test=set(interactions_test_df['userCode'])
x_train=set(interactions_train_df['userCode'])

only_in_set1 = x_train - x_test


#print("Elements in train but not in test:", only_in_set1)

only_in_set2 = x_test - x_train

#print("Elements in test but not in train:", only_in_set2)

In [ ]:
#Creating a sparse pivot table with users in rows and items in columns
items_users_pivot_matrix_df = interactions_train_df.pivot(index='userCode',
                                                          columns='name_encoded',
                                                          values='price').fillna(0)

In [ ]:
items_users_pivot_matrix = items_users_pivot_matrix_df.values
#items_users_pivot_matrix[:10]

user_ids = list(items_users_pivot_matrix_df.index)
#user_ids[:10]

In [ ]:
#items_users_pivot_matrix.shape

# The number of factors to factor the item-user matrix.
NUMBER_OF_FACTORS_MF = 8

In [ ]:
import scipy
from scipy.sparse.linalg import svds
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(items_users_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [ ]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = items_users_pivot_matrix_df.columns,index=user_ids).transpose()
#cf_preds_df.head()

In [ ]:
class CFRecommender:

    MODEL_NAME = 'Collaborative Filtering'

    def __init__(self, cf_predictions_df , items_df):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

In [ ]:
def recommend_items(self, user_id, items_to_ignore=[], topn=5, verbose=False):
        if user_id not in self.cf_predictions_df.columns:
            raise KeyError(f"User '{user_id}' not found in prediction data.")
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['name_encoded'].isin(items_to_ignore)].sort_values('recStrength', ascending=False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            # Merge recommendations_df with items_df
            recommendations_df = recommendations_df.merge(self.items_df, how='left',
                                                          left_on='name_encoded',
                                                          right_on='name_encoded')[['name_encoded','name','recStrength']]
            recommendations_df=pd.DataFrame(recommendations_df.groupby('name').max('recStrength').sort_values('recStrength', ascending=False))

        return recommendations_df

In [ ]:
# Assuming cf_preds_df and interactions_from_selected_users_df are defined elsewhere
cf_recommender_model = CFRecommender(cf_preds_df, interactions_from_selected_users_df)

#with open('cf_recommender_model.pkl', 'wb') as f:
    #pickle.dump(cf_recommender_model, f)

#cf_recommender_model.recommend_items(590)

In [ ]:
def main():
    st.title('Hotel Recommendation App')


    # Create dropdowns for Package Type, Start City, and Destination
    usercode = hotel_df['userCode'].unique()


    usercode = st.selectbox('Select usercode:', usercode)
    #Hotel_Info = st.selectbox('Select Hotel_Info:', Hotel_Info)
    #price = st.selectbox('Select Hotel_Info:', price)

    recommended_hotels = 0.0 # Default value
# Slider for Price
    #price = st.slider('Select Maximum Price:', min_value=0.00, max_value=float(data['price'].max()))


    # Get recommendations on button click
    if st.button('Get Recommendations'):
        recommended_hotels = cf_recommender_model.recommend_items(usercode,verbose=True)
        if isinstance(recommended_hotels, str):
            st.warning(recommended_hotels)
        else:
            st.table(recommended_hotels)

In [ ]:
if __name__ == '__main__':
    main()

2024-04-21 10:45:57.596 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
